In [ ]:
import os
import numpy as np

import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
# import seaborn as sns
# sns.set_theme(rc={'figure.figsize':(18,9)})
import plotly.express as px
pd.options.plotting.backend = "plotly"

### Load B3 Setor

In [ ]:
setorial_path = os.path.join("data", "b3_setorial.csv")
df_setorial = pd.read_csv(
    setorial_path, sep=";", encoding="latin"
)

In [ ]:
df_setorial.sample(n=5)

### Ações

In [ ]:
df_tmp = pd.read_excel('data/posicao-2023-01-27.xlsx', sheet_name='Acoes')

In [ ]:
df = df_tmp.copy()

df.drop(columns=['Conta', 'Código ISIN / Distribuição', 'Escriturador', 
    'Quantidade Disponível', 'Quantidade Indisponível', 
    'Motivo', 'Preço de Fechamento'], inplace=True)

df.rename(
    columns={
        'Produto' : 'des_produto',
        'Instituição' : 'des_conta',
        'Código de Negociação' : 'des_movimentacao',
        'Código de Negociação' : 'cod_acao',
        'Tipo' : 'tp_acao',
        'Quantidade' : 'quantidade',
        'Valor Atualizado' : 'vlr_total'
    }, 
    inplace=True
)

df = df[(~df['des_produto'].isna()) & (~df['des_conta'].isna())]

In [ ]:
df['des_produto'] = df['des_produto'].str.lstrip().str.rstrip()
df['des_produto'] = df['des_produto'].str[7:]
df['des_produto'] = df['des_produto'].str.replace('- TRANSMISSORA', 'TRANSMISSORA')
df['vlr_total'] = df['vlr_total'].astype(np.float32)

In [ ]:
# Merge Setor
df['tmp_cod_acap'] = df['cod_acao'].str[0:4]
df = pd.merge(df, df_setorial, left_on='tmp_cod_acap', right_on='codigo')

In [ ]:
# Percentual da acao na carteira
df['tmp_vlr_total'] = df['vlr_total'].sum()
df['pct_vlr_total'] = (df['vlr_total'] / df['tmp_vlr_total']) * 100

In [ ]:
df_setor = df[['setor','vlr_total']].groupby(['setor']).sum().reset_index().rename(columns={'vlr_total' : 'vlr_total_setor'})
def_subsetor = df[['subsetor','vlr_total']].groupby(['subsetor']).sum().reset_index().rename(columns={'vlr_total' : 'vlr_total_subsetor'})
df_segmento = df[['segmento','vlr_total']].groupby(['segmento']).sum().reset_index().rename(columns={'vlr_total' : 'vlr_total_segmento'})

df = pd.merge(df, df_setor, on='setor')
df = pd.merge(df, def_subsetor, on='subsetor')
df = pd.merge(df, df_segmento, on='segmento')

In [ ]:
# Percentual do setor/subsetor/segmento da carteira
df['pct_vlr_total_setor'] = (df['vlr_total_setor'] / df['tmp_vlr_total']) * 100
df['pct_vlr_total_subsetor'] = (df['vlr_total_subsetor'] / df['tmp_vlr_total']) * 100
df['pct_vlr_total_segmento'] = (df['vlr_total_segmento'] / df['tmp_vlr_total']) * 100

In [ ]:
columns = ['des_conta', 'cod_acao', 'des_produto', 
        'tp_acao', 'setor', 'subsetor', 'segmento',
        'quantidade', 'vlr_total', 'pct_vlr_total',
        'vlr_total_setor', 'pct_vlr_total_setor',
        'vlr_total_subsetor', 'pct_vlr_total_subsetor',
        'vlr_total_segmento', 'pct_vlr_total_segmento']
        
df = df[columns]

In [ ]:
px.treemap(
    df, 
    path=["setor", "subsetor", "segmento", "cod_acao"], 
    values="pct_vlr_total",
    width=1000, height=800
    )

### FII

In [ ]:
df_tmp_fundos = pd.read_excel('data/posicao-2023-01-27.xlsx', sheet_name='Fundo de Investimento')

In [ ]:
df_fundos = df_tmp_fundos.copy()

In [ ]:
df_fundos.drop(columns=['Conta', 'Código ISIN / Distribuição', 'Administrador', 
    'Quantidade Disponível', 'Quantidade Indisponível', 'tp_acao'
    'Motivo', 'Preço de Fechamento'], inplace=True)

df_fundos.rename(
    columns={
        'Produto' : 'des_produto',
        'Instituição' : 'des_conta',
        'Código de Negociação' : 'des_movimentacao',
        'Código de Negociação' : 'cod_acao',
        'Tipo' : 'tp_acao',
        'Quantidade' : 'quantidade',
        'Valor Atualizado' : 'vlr_total'
    }, 
    inplace=True
)

df_fundos = df_fundos[(~df_fundos['des_produto'].isna()) & (~df['des_conta'].isna())]

In [ ]:
df_fundos['vlr_total'] = df_fundos['vlr_total'].astype(np.float32)

# Percentual da acao na carteira
df_fundos['tmp_vlr_total'] = df_fundos['vlr_total'].sum()
df_fundos['pct_vlr_total'] = (df_fundos['vlr_total'] / df_fundos['tmp_vlr_total']) * 100

In [ ]:
df_fundos

In [ ]:
tp_fii = {
    'BTLG11' : 'Tijolo',
    'KNRI11' : 'Tijolo',
    'XPML11' : 'Tijolo',
    'KNCA11' : 'Papel',
    'OUJP11' : 'Papel'
}

df_fundos['tp_fii'] = df_fundos['cod_acao'].map(tp_fii)

In [ ]:
segmento = {
    'BTLG11' : 'Logística',
    'KNRI11' : 'Híbrido',
    'XPML11' : 'Shoppings',
    'KNCA11' : 'Títulos e Valores Mobiliários',
    'OUJP11' : 'Títulos e Valores Mobiliários'
}

df_fundos['segmento'] = df_fundos['cod_acao'].map(segmento)

In [ ]:
df_fundos

In [ ]:
px.treemap(
    df_fundos, 
    path=["tp_fii", "segmento", "cod_acao"], 
    values="pct_vlr_total",
    width=1000, height=800
    )

### Tesouro Direto

In [ ]:
df_tmp_tesouro_direto = pd.read_excel('data/posicao-2023-01-27.xlsx', sheet_name='Tesouro Direto')

In [ ]:
df_tesouro_direto = df_tmp_tesouro_direto.copy()

df_tesouro_direto.drop(columns=['Instituição', 'Código ISIN', 'Quantidade Disponível', 
    'Quantidade Indisponível', 'Motivo', 'Valor Atualizado'], inplace=True)

df_tesouro_direto.rename(
    columns={
        'Produto' : 'des_produto',
        'Indexador' : 'indexador',
        'Vencimento' : 'vencimento',
        'Quantidade' : 'quantidade',
        'Valor Aplicado' : 'vlr_aplicado',
        'Valor bruto' : 'vlr_bruto',
        'Valor líquido' : 'vlr_liquido'
    }, 
    inplace=True
)

df_tesouro_direto = df_tesouro_direto[(~df_tesouro_direto['des_produto'].isna()) & (~df_tesouro_direto['indexador'].isna())]

In [ ]:
# Percentual da acao na carteira
df_tesouro_direto['tmp_vlr_liquido_total'] = df_tesouro_direto['vlr_liquido'].sum()
df_tesouro_direto['pct_vlr_liquido_total'] = (df_tesouro_direto['vlr_liquido'] / df_tesouro_direto['tmp_vlr_liquido_total']) * 100

In [ ]:
df_tesouro_direto

In [ ]:
px.treemap(
    df_tesouro_direto, 
    path=["indexador", "des_produto"], 
    values="pct_vlr_liquido_total",
    width=1000, height=800
    )